In [ ]:
import networkx as nx
import pandas as pd
import os
import numpy as np
import sys
import math

create adjacent matrices for nets

In [ ]:
def netmatize(net):
    n=net.max().max()
    Net=np.zeros((n,n))
    N=net.shape[0]
    for i in range(N):
        x=net.iloc[i,0]
        y=net.iloc[i,1]
        Net[x,y]=Net[x,y]+1
    return Net 

In [ ]:
for root, dirs, files in os.walk("./travian-dataset/Attacks-Network-CSV"):  
    print('extracting attack')
files.sort()

In [ ]:
i=0
for file in files:
    filepath="./travian-dataset/Attacks-Network-CSV/"+file
    globals()["attack"+str(i)] = netmatize(pd.read_csv(filepath,header=None,usecols=[1,2]))
    i=i+1

In [ ]:
for root, dirs, files in os.walk("./travian-dataset/Messages-Network-CSV"):  
    print('extracting message')
files.sort()

In [ ]:
i=0
for file in files:
    filepath="./travian-dataset/Messages-Network-CSV/"+file
    globals()["message"+str(i)] = netmatize(pd.read_csv(filepath,header=None,usecols=[1,2]))
    i=i+1

In [ ]:
for root, dirs, files in os.walk("./travian-dataset/Trades-Network-CSV"):  
    print('extracting trade')
files.sort()

In [ ]:
i=0
for file in files:
    filepath="./travian-dataset/Trades-Network-CSV/"+file
    globals()["trade"+str(i)] = netmatize(pd.read_csv(filepath,header=None,usecols=[1,2]))
    i=i+1

In [ ]:
for root, dirs, files in os.walk("./travian-dataset/Communities"):  
    print('extracting community')
files.sort()

In [ ]:
def matize(Community):
    n=max(Community)
    Com=np.zeros((n,n))
    for i in Community:
        for j in i:
            for jj in i:
                Com[j,jj]=1
    return Com  

In [ ]:
i=0
for file in files:
    filepath="./travian-dataset/communities/"+file
    result=[]
    with open(filepath,'r') as f:
        for line in f:
            result.append(list(map(int,line.strip('\n').split(' '))))
    globals()["community"+str(i)] = result
    i=i+1

louvain method

In [ ]:
def k(i):
    return A[:,i].sum()

In [ ]:
def Modularity(A,C):
    #A==adjecent matrix; C==labeling
    n=A.shape[0]
    m=A.sum()
    Q=0
    for i in range(n):
        for j in range(n):
            Q=Q+(A[i,j]-k(i)*k(j)/m)*(C[i]==C[j])
    Q=Q/m
    return Q

In [ ]:
def louvain(A,lowerboundofnum,lowerboundofdiff):
    diffmax=lowerboundofdiff+1e-5
    n=A.shape[0]
    C=range(n)
    while(n>lowerbound and diffmax>lowerboundofdiff):
        diff=[]
        Q=[]
        for i in range(n):
            Q0=Modularity(A,C)
            nb=np.argwhere(A.tolist()[i]>0)
            for j in nb:
                C1=C
                C1[i]=C1[j]
                Q1=Modularity(A,C1)
                diff=diff.append(Q1-Q0)
                if diff.max>lowerboundofdiff:
                    I=diff.index(diff.max())
                    C[i]=C[I]
                if diff.max<diffmax:
                    diffmax=diff.max
        n=len(set(C))
    return C

The similarity of true community and the result

In [ ]:
def similarity(Com,C):
    n=Com.shape[0]
    Comnew=np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if C[i]==C[j]:
                Comnew[i,j]=1
    return (Com==Comnew).mean()

testing

In [ ]:
acca=[]
accm=[]
acct=[]
for i in range(1,30):
    N=len(globals()["community"+str(i)])
    Com=matize(globals()["community"+str(i)])
    A=louvain(globals()["attack"+str(i)],N,1e-5)
    acca=acca.append(similarity(Com,A))
    M=louvain(globals()["massage"+str(i)],N,1e-5)
    accm=accm.append(similarity(Com,M))
    T=louvain(globals()["trade"+str(i)],N,1e-5)
    acct=acct.append(similarity(Com,T))

In [ ]:
print(mean(acca))
print(mean(acct))
print(mean(accm))